# Structured Q&A

Source code: https://github.com/mozilla-ai/structured-qa

Docs: https://mozilla-ai.github.io/structured-qa

## Installing dependencies

In [1]:
!git clone --single-branch --branch 5-add-benchmark https://github.com/mozilla-ai/structured-qa

Cloning into 'structured-qa'...
remote: Enumerating objects: 803, done.
remote: Counting objects: 100% (241/241), done.
remote: Compressing objects: 100% (137/137), done.
remote: Total 803 (delta 154), reused 133 (delta 97), pack-reused 562 (from 1)
Receiving objects: 100% (803/803), 2.29 MiB | 28.93 MiB/s, done.
Resolving deltas: 100% (436/436), done.


In [2]:
%pip install ./structured-qa

Processing ./structured-qa
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 8.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 63.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 68.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 68.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 61.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 4.1 MB/s eta 0:00:00
  Created wheel for structured-qa: filename=structured_qa-0.3.3.dev86+gb726447-py3-none-any.whl size=16326 sha256=dd9419f56083c11c4717d20ca525f0f2ec3d4d42390326c036c361d00

In [3]:
%pip install --quiet https://github.com/abetlen/llama-cpp-python/releases/download/v0.3.4-cu122/llama_cpp_python-0.3.4-cp311-cp311-linux_x86_64.whl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 445.2/445.2 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.2 MB/s eta 0:00:00


# Setup

In [4]:
import os

os.environ["LOGURU_LEVEL"] = "INFO"

In [5]:
from loguru import logger

## Function to Process all questions for a single Section

In [6]:
import time


ANSWER_WITH_TYPE_PROMPT = """
You are a rigorous assistant answering questions.
You only answer based on the current information available.
The current information available is:

```
{CURRENT_INFO}
```

If the current information available not enough to answer the question,
you must return the following message and nothing else:

```
I need more info.
```

The answer must be in one of the following formats:
- YES/NO (for boolean questions)
Question: Is the model an LLM?
Answer: YES
- Number (for numeric questions)
Question: How many layers does the model have?
Answer: 12
- Single letter (for multiple-choice questions)
Question: What is the activation function used in the model? -A: ReLU -B: Sigmoid -C: Tanh
Answer: C
"""


def process_section_questions(
    section_file,
    section_data,
    model,
):
    logger.info("Predicting")
    answers = {}
    sections = {}
    for index, row in section_data.iterrows():
        question = row["question"]
        logger.info(f"Question: {question}")
        messages = [
            {
                "role": "system",
                "content": ANSWER_WITH_TYPE_PROMPT.format(
                    CURRENT_INFO=section_file.read_text()
                ),
            },
            {"role": "user", "content": question},
        ]
        response = model.get_response(messages)
        logger.info(f"Answer: {response}")
        answers[index] = response
        sections[index] = None
    return answers, sections

## Load Model

In [7]:
from structured_qa.model_loaders import load_llama_cpp_model

In [8]:
model = load_llama_cpp_model(
    "bartowski/Qwen2.5-7B-Instruct-GGUF/Qwen2.5-7B-Instruct-Q8_0.gguf"
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Qwen2.5-7B-Instruct-Q8_0.gguf:   0%|          | 0.00/8.10G [00:00<?, ?B/s]

# Run Benchmark

In [9]:
from pathlib import Path

import pandas as pd


logger.info("Loading input data")
data = pd.read_csv("structured-qa/benchmark/structured_qa.csv")
data["pred_answer"] = [None] * len(data)
data["pred_section"] = [None] * len(data)

for section_name, section_data in data.groupby("section"):
    section_file = Path(f"structured-qa/benchmark/perfect_context/{section_name}.txt")

    answers, sections = process_section_questions(section_file, section_data, model)

    for index in section_data.index:
        data.loc[index, "pred_answer"] = str(answers[index]).upper()
        data.loc[index, "pred_section"] = sections[index]

data.to_csv("results.csv")

2025-01-30 10:01:01.101 | INFO     | __main__:<cell line: 0>:6 - Loading input data
2025-01-30 10:01:01.135 | INFO     | __main__:process_section_questions:38 - Predicting
2025-01-30 10:01:01.139 | INFO     | __main__:process_section_questions:43 - Question: In billions, how many trainable parameters does GPT-3 have?
2025-01-30 10:01:02.914 | INFO     | __main__:process_section_questions:54 - Answer: 175
2025-01-30 10:01:02.916 | INFO     | __main__:process_section_questions:43 - Question: Does LoRA introduce additional inference latency compared to full fine-tuning?
2025-01-30 10:01:03.032 | INFO     | __main__:process_section_questions:54 - Answer: NO
2025-01-30 10:01:03.038 | INFO     | __main__:process_section_questions:38 - Predicting
2025-01-30 10:01:03.040 | INFO     | __main__:process_section_questions:43 - Question: What fire resistance must vertical partitions have? -A: EI30 -B: EI60 -C: EI90
2025-01-30 10:01:03.638 | INFO     | __main__:process_section_questions:54 - Answer:

# Results

In [18]:
results = pd.read_csv("results.csv")
for index, result in results.iterrows():
  if result["pred_answer"].startswith((f"-{result['answer']}:", f"{result['answer']}")):
    results.loc[index, "pred_answer"] = result["answer"]
results.loc[results["answer"] != results["pred_answer"]]

,Unnamed: 0,document,section,question,answer,pred_answer,pred_section
10,10,https://arxiv.org/pdf/1706.03762,5.4 Regularization,What was the dropout rate used for the base mo...,0.1,PDROP = 0.1,NaN
14,14,https://arxiv.org/pdf/2210.05189,3 Experimental Results,How many parameters are in the toy model (y = ...,14,NUMBER\nQUESTION: HOW MANY PARAMETERS ARE IN T...,NaN
38,38,https://arxiv.org/pdf/2201.11903,3.1 Experimental Setup,How many large language models were evaluated?,5,FIVE,NaN
43,43,https://arxiv.org/pdf/2201.11903,3.4 Robustness of Chain of Thought,How many annotators provided independent chain...,3,2,NaN
47,47,https://github.com/mozilla-ai/structured-qa/re...,CARD AND TILE EFFECTS,How many different races are there?,6,5,NaN
51,51,https://github.com/mozilla-ai/structured-qa/re...,CHAPTER OVERVIEW,"After taking a landmark tile, do you reveal a ...",NO,YES,NaN
52,52,https://github.com/mozilla-ai/structured-qa/re...,CARD AND TILE COSTS,Can a player pay coins to compensate for missi...,YES,NO,NaN
65,65,https://github.com/mozilla-ai/structured-qa/re...,EXPEDITION PHASE,Do you need a fish to conquer a distant island?,YES,NO,NaN
78,78,https://docs.nvidia.com/cuda/pdf/CUDA_C_Progra...,5.2. Thread Hierarchy,Can you identify a thread with a four-dimensio...,NO,I NEED MORE INFO.,NaN


In [19]:
accuracy = sum(results["answer"] == results["pred_answer"]) / len(results)
accuracy

0.9090909090909091